In [1]:
import os

import Mux_selector
import Mux_input_parser

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

2023-03-25 08:57:11.394836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 08:57:14.480143: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-25 08:57:14.480338: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-25 08:57:14.480362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
Task_set = [
    'translate_de_en',
    'translate_en_de',
    'BoolQ',
    'TriviaQA',
    'cola',
    'squad_v2',
    "nq_open",
    'rte',
    'mnli',
    'mrpc',
    'qnli',
]

In [3]:
HOSOTA = {}

In [4]:
# Closed Book QA
t5_qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-small-ssm-nq")
t5_tok = AutoTokenizer.from_pretrained("google/t5-small-ssm-nq")

def predict_trivia(s):
    input_ids = t5_tok(s, return_tensors="pt").input_ids
    gen_output = t5_qa_model.generate(input_ids)[0]
    return t5_tok.decode(gen_output, skip_special_tokens=True)

HOSOTA['TriviaQA'] = predict_trivia

In [5]:
# BoolQ
t5_qa_model_boolq = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-small-finetuned-boolq")
t5_tok_boolq = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-boolq")

def predict_boolq(s):
    input_ids = t5_tok_boolq(s, return_tensors="pt").input_ids
    gen_output = t5_qa_model_boolq.generate(input_ids)[0]
    return t5_tok_boolq.decode(gen_output, skip_special_tokens=True)

HOSOTA['BoolQ'] = predict_boolq

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [7]:
def inference(trained_model, trained_tokenizer, text):
    generator = pipeline(
        "text2text-generation",
        model=trained_model,
        tokenizer=trained_tokenizer,
        device=0 if torch.cuda.is_available() else -1, # use GPU if available
    )

    input_text = text # input text for generation
    generated_text = generator(input_text, max_length=50)
    print(generated_text)

In [8]:
output_dir="HoSoTAs/en-de-new"
path = os.path.join(output_dir, "model_files")
trained_tokenizer = T5Tokenizer.from_pretrained(path)
trained_model = T5ForConditionalGeneration.from_pretrained(path)

def translate(s):
    s = Mux_input_parser.get_summarized_string(s)  
    return inference(trained_model=trained_model, trained_tokenizer=trained_tokenizer, text=s)

HOSOTA['translate_en_de'] = translate

In [9]:
s = "When was Lincoln born?"
s = "True or False: sun is yellow"
s = "Translate Machine learning is great, isn't it? from english to german"

In [10]:
def mux(s):
    model_name =  Mux_selector.predict(s)
    print("Task: ", model_name)
    model = HOSOTA[model_name]
    return model(s)

In [11]:
mux(s)

Task:  translate_en_de


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3712: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[{'generated_text': 'Das Maschinelle Lernen ist großartig, ist es nicht?'}]


In [12]:
While True:
    s = input("enter: ")
    print(mux(s))

SyntaxError: invalid syntax (3299829458.py, line 1)